# Implementation of Simple Moving Average (SMA) Crossover Strategy
inspired by shreyas sarda

In [1]:
# import libraries
import datetime
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
plt.style.use('fivethirtyeight')

We use pandas_datareader library to import data from Yahoo Finance

In [2]:
from pandas_datareader import data as web

import yfinance as yf
yf.pdr_override()

# Input the data
stock = input('Stock name is: ')
num_days = int(input('Number of days the analysis needs to be done(Please enter an integer): '))

# Set start date of analysis
start_date = (datetime.datetime.now() - datetime.timedelta(days = num_days)).strftime("%m-%d-%Y")

# Import Data from Yahoo Finance
df = web.DataReader(stock, data_source = 'yahoo', start = start_date)

# Remove rows having np.NaN
df.dropna(how = 'any', inplace = True)

Stock name is: PLTR
Number of days the analysis needs to be done(Please enter an integer): 1500


TypeError: download() got an unexpected keyword argument 'data_source'